In [1]:
#unzip a file.
#from zipfile import ZipFile
#zip = ZipFile('devanagari+handwritten+character+dataset.zip')
#zip.extractall()

In [70]:
import pandas as pd

import numpy as np

import matplotlib.pyplot as plt

import os
import scipy.stats as s

In [43]:
from sklearn.metrics import classification_report

In [44]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, roc_curve, average_precision_score, recall_score

In [45]:
train_path = "./DevanagariHandwrittenCharacterDataset/Train"

test_path = "./DevanagariHandwrittenCharacterDataset/Test"

In [46]:
unique_labels = os.listdir(train_path)

In [47]:
def list_of_images(folder):
    
    return os.listdir(os.path.join(train_path,folder))

In [48]:
def read_image(folder,image):
    
    folder_path = os.path.join(train_path,folder)
    
    image_path = os.path.join(folder_path,image)
    
    image = plt.imread(image_path)
    
    return image.reshape(image.shape[0]*image.shape[1],)

In [49]:
def stacking_row_vectors(folder):
    
    images_list = list_of_images(folder)
    
    images = []
    
    for img in images_list:
        
        images.append(read_image(folder,img))
        
    return np.array(images[:])

In [50]:
train_data = []

for folder in unique_labels:
    
    train_folder_matrix= stacking_row_vectors(folder)
    
    train_data.append(train_folder_matrix)
    
train_data = np.concatenate(train_data,axis=0)

In [59]:
for i in unique_labels:
    m=list_of_images(i)    
train_labels = []
for folder_name in unique_labels:
    
    train_labels = train_labels + [folder_name]*len(m)

In [63]:
train_data = pd.DataFrame(data=train_data)
train_data['label'] = train_labels
train_data

,0,1,2,3,4,5,6,7,8,9,...,1015,1016,1017,1018,1019,1020,1021,1022,1023,label
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,character_10_yna
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,character_10_yna
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,character_10_yna
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,character_10_yna
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,character_10_yna
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,digit_9
78196,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,digit_9
78197,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,digit_9
78198,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,digit_9


In [64]:
from sklearn.metrics import classification_report,accuracy_score,precision_score,recall_score

In [65]:
class GaussianNB:
    
    """Instantiate a Gaussian Naive Bayes Object with the following parameters: 
        
        features :               A dataframe consisting of continuous features, excluding labels
        labels :                 A series consisting of binary labels
        train_cv_test_split :    A tuple consisting of fraction for training, cross validation and testing data
        apply_pca :              Boolean value specifying whether to apply PCA or not
        n_principal_components : Number of Principal Components (Eigen vectors having non zero values to keep) 
    """
    
    def __init__(self,features,labels,train_cv_test_split,apply_pca,n_principal_components):
        
        self.unique_labels = list(labels.unique())
        
        self.labels = np.array(labels).reshape(labels.shape[0],1)
        
        self.train_cv_test_split = train_cv_test_split
        
        self.n_principal_components = n_principal_components
        
        if apply_pca == True:
            
            self.X_new = self.apply_dim_reduction(features,self.n_principal_components)
            
            
            
            
    def apply_dim_reduction(self,data,n_components):
        
        X = np.array(data)
        
        X_dash = X - np.mean(X,axis=0).reshape(-1,X.shape[1])
        
        sigma_hat = (1/data.shape[0])*np.matmul(X_dash.T,X_dash)
        
        sigma_hat_decompose = np.linalg.svd(sigma_hat)
        
        Q = sigma_hat_decompose[0]
        
        self.Q_tilda = Q[:,0:n_components]
        
        X_new = np.matmul(X_dash,self.Q_tilda)
        
        return X_new
    
    
    
    
    def fit(self,data,alpha,gamma):
        
        self.likelihood_params = dict()
        
        sigma_hats = 0
        
        for label in self.unique_labels:
        
            mu_hat = np.array(data[data['label'] == label].iloc[:,0:self.n_principal_components].mean())

            sigma_hat = np.array(data[data['label'] == label].iloc[:,0:self.n_principal_components].cov())
            
            sigma_hats = sigma_hats + 1359*sigma_hat
            
            self.likelihood_params[label] = [mu_hat,sigma_hat]
            
        self.sigma_hat = sigma_hats/(data.shape[0] - len(self.unique_labels))
        
        self.mean_variance = np.mean(np.diag(self.sigma_hat))
            
        for label in self.unique_labels:
            
            self.likelihood_params[label][1] = alpha*self.likelihood_params[label][1] + (1-alpha)*self.sigma_hat
            
            self.likelihood_params[label][1] = (1-gamma)*self.likelihood_params[label][1] + gamma*self.mean_variance*np.eye(self.n_principal_components,self.n_principal_components)
        
        
        
        
    def evaluate(self,data):
        
        inputs = np.array(data.iloc[:,0:self.n_principal_components])
        
        posterior = list()
        
        for label in self.unique_labels:
    
            posterior.append(s.multivariate_normal.pdf(inputs,self.likelihood_params[label][0],self.likelihood_params[label][1]).reshape(inputs.shape[0],1))
        
        posterior = np.concatenate(posterior,axis=1)
        
        predicted_category = pd.Series(np.argmax(posterior,axis=1))
    
        predicted_category.replace(to_replace=np.arange(0,len(self.unique_labels)),value=self.unique_labels,inplace=True)
    
        predicted_results = np.array(predicted_category)
        
        actual_results = np.array(data['label'])
        
        acc = accuracy_score(y_true=actual_results,y_pred=predicted_results)
        
        recall = recall_score(y_true=actual_results,y_pred=predicted_results,average='weighted')
        
        precision = precision_score(y_true=actual_results,y_pred=predicted_results,average='weighted')
        
        return {"acc":acc,"recall":recall,"precision":precision}



In [71]:
D = dict()

for n_components in [50,100,150,200,250,300,350,400,450,500,550,600,650,700]:
    
    obj = GaussianNB(features=train_data.iloc[:,0:1024],labels=train_data.iloc[:,1024],train_cv_test_split=(0.7,0.2,0.1),
                             apply_pca=True,n_principal_components=n_components)

    X_train = pd.DataFrame(obj.X_new)

    X_train['label'] = train_data['label']

    obj.fit(X_train,0.5,0.3)

    X_cv = np.matmul(np.array(cv_data.iloc[:,0:1024]),obj.Q_tilda)

    X_cv = pd.DataFrame(data=X_cv)

    X_cv['label'] = cv_data['label']

    D[n_components] = obj.evaluate(X_cv)

In [72]:
D

{50: {'acc': 0.7761508951406649,
  'recall': 0.7761508951406649,
  'precision': 0.8377394209556167},
 100: {'acc': 0.8604859335038363,
  'recall': 0.8604859335038363,
  'precision': 0.8868966118797517},
 150: {'acc': 0.8917519181585678,
  'recall': 0.8917519181585678,
  'precision': 0.9080181538887908},
 200: {'acc': 0.9097186700767264,
  'recall': 0.9097186700767264,
  'precision': 0.9207620126014018},
 250: {'acc': 0.9218030690537085,
  'recall': 0.9218030690537085,
  'precision': 0.9297121013585642},
 300: {'acc': 0.9310102301790282,
  'recall': 0.9310102301790282,
  'precision': 0.9370104994738511},
 350: {'acc': 0.9374680306905371,
  'recall': 0.9374680306905371,
  'precision': 0.9420947922474718},
 400: {'acc': 0.9432225063938618,
  'recall': 0.9432225063938618,
  'precision': 0.9470414686964109},
 450: {'acc': 0.9478260869565217,
  'recall': 0.9478260869565217,
  'precision': 0.950964052357067},
 500: {'acc': 0.9517902813299233,
  'recall': 0.9517902813299233,
  'precision': 0.9